In [ ]:
import sys

import rdkit # required for cheminformatics functionality (small molecules)
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs

import pandas as pd
import numpy as np

In [2]:
# load the molecules
with open("../data/new_descriptors/metabolite_id_classes.txt") as f:
    mols_str, mols_id, classes = zip(*[ line.strip().split() for line in f])

## 1. MACCS Distinct

In [3]:
maccs_distinct_desc = pd.read_csv('../data/new_descriptors/maccs_distinct_desc.txt', names=["fingerprint", "mol_id"], delimiter=" ")

In [4]:
maccs_distinct_desc_data = []

for i in range(len(maccs_distinct_desc['fingerprint'])):
    maccs_distinct_desc_data.append(np.array([int(j) for j in list(maccs_distinct_desc['fingerprint'].iloc[i])]).T)

In [5]:
maccs_distinct_desc_labels = []

for i in range(len(maccs_distinct_desc['mol_id'])):
    
    isFound = False
    
    if not "mol" in maccs_distinct_desc['mol_id'].iloc[i]:
        maccs_distinct_desc['mol_id'].iloc[i] = "mol_" + maccs_distinct_desc['mol_id'].iloc[i]
    
    for j in range(len(mols_id)):
        if maccs_distinct_desc['mol_id'].iloc[i] == mols_id[j]:
            maccs_distinct_desc_labels.append(classes[j])
            isFound = True
            break
            
    if isFound == False:
        print(maccs_distinct_desc['mol_id'].iloc[i])

In [6]:
# Convert classes to multi-class vector

multi_class_output = []

for elem in maccs_distinct_desc_labels:
    
    my_class = np.zeros(11)
    
    for num in elem.split(","):
        my_class[int(num)] = 1
        
    multi_class_output.append(my_class)

In [7]:
multi_class_output = np.array(multi_class_output)

In [8]:
np.sum(multi_class_output, axis=0)

array([133.,  79., 155.,  80., 338., 122.,  24., 243., 381., 529., 569.])

In [9]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.preprocessing import MinMaxScaler

In [10]:
scaler = MinMaxScaler()

maccs_distinct_desc_data = scaler.fit_transform(maccs_distinct_desc_data)

X_train, X_test, y_train, y_test = train_test_split(maccs_distinct_desc_data, multi_class_output, test_size=0.2)


In [11]:
np.sum(y_train, axis=0)

array([103.,  59., 123.,  69., 272., 102.,  20., 195., 304., 430., 443.])

In [12]:
np.sum(y_test, axis=0)

array([ 30.,  20.,  32.,  11.,  66.,  20.,   4.,  48.,  77.,  99., 126.])

In [13]:
np.sum(y_test, axis=0)/np.sum(y_train, axis=0)

array([0.29126214, 0.33898305, 0.2601626 , 0.15942029, 0.24264706,
       0.19607843, 0.2       , 0.24615385, 0.25328947, 0.23023256,
       0.28442438])

In [14]:
from skmultilearn.model_selection import iterative_train_test_split

X_train, y_train, X_test, y_test = iterative_train_test_split(maccs_distinct_desc_data, multi_class_output, test_size = 0.2)



In [15]:
np.sum(y_train, axis=0)

array([106.,  63., 124.,  64., 270.,  98.,  19., 194., 305., 423., 455.])

In [16]:
np.sum(y_test, axis=0)

array([ 27.,  16.,  31.,  16.,  68.,  24.,   5.,  49.,  76., 106., 114.])

In [17]:
np.sum(y_test, axis=0)/np.sum(y_train, axis=0)

array([0.25471698, 0.25396825, 0.25      , 0.25      , 0.25185185,
       0.24489796, 0.26315789, 0.25257732, 0.24918033, 0.25059102,
       0.25054945])

In [28]:
clf = RandomForestClassifier(n_estimators=500, min_samples_split=5, max_depth=10, verbose=True)
clf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    1.4s finished


RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=500,
                       verbose=True)

In [29]:
y_pred = clf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.2s finished


In [30]:
y_pred[y_pred<0.5]  = 0
y_pred[y_pred>=0.5] = 1

In [31]:
for c in range(11):
    y_true_c = y_test[:,c]
    y_pred_c = y_pred[:,c]
    
    auc       = accuracy_score(y_true_c, y_pred_c)
    precision = precision_score(y_true_c, y_pred_c)
    recall    = recall_score(y_true_c, y_pred_c)
    
    print('Class '+str(c+1)+' statistics:')
    print('Accuracy %.4f, Precision %.4f, Recall %.4f\n' %(auc, precision, recall))

Class 1 statistics:
Accuracy 0.9468, Precision 1.0000, Recall 0.1481

Class 2 statistics:
Accuracy 0.9630, Precision 0.0000, Recall 0.0000

Class 3 statistics:
Accuracy 0.9491, Precision 0.9091, Recall 0.3226

Class 4 statistics:
Accuracy 0.9630, Precision 0.5000, Recall 0.0625

Class 5 statistics:
Accuracy 0.8634, Precision 1.0000, Recall 0.1324

Class 6 statistics:
Accuracy 0.9468, Precision 1.0000, Recall 0.0417

Class 7 statistics:
Accuracy 0.9884, Precision 0.0000, Recall 0.0000

Class 8 statistics:
Accuracy 0.9259, Precision 0.9048, Recall 0.3878

Class 9 statistics:
Accuracy 0.9190, Precision 0.8727, Recall 0.6316

Class 10 statistics:
Accuracy 0.8796, Precision 0.9355, Recall 0.5472

Class 11 statistics:
Accuracy 0.8681, Precision 0.9130, Recall 0.5526



/Users/gianluca/python_venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gianluca/python_venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 2. Morgan distinct

In [32]:
morgan_distinct_desc = pd.read_csv('../data/new_descriptors/morgan_distinct_desc.txt', names=["fingerprint", "mol_id"], delimiter=" ")

In [33]:
morgan_distinct_desc_data = []

for i in range(len(morgan_distinct_desc['fingerprint'])):
    morgan_distinct_desc_data.append(np.array([int(j) for j in list(morgan_distinct_desc['fingerprint'].iloc[i])]).T)

In [34]:
morgan_distinct_desc_labels = []

for i in range(len(morgan_distinct_desc['mol_id'])):
    
    isFound = False
    
    if not "mol" in morgan_distinct_desc['mol_id'].iloc[i]:
        morgan_distinct_desc['mol_id'].iloc[i] = "mol_" + morgan_distinct_desc['mol_id'].iloc[i]
    
    for j in range(len(mols_id)):
        if morgan_distinct_desc['mol_id'].iloc[i] == mols_id[j]:
            morgan_distinct_desc_labels.append(classes[j])
            isFound = True
            break
            
    if isFound == False:
        print(morgan_distinct_desc['mol_id'].iloc[i])

In [35]:
# Convert classes to multi-class vector

multi_class_output = []

for elem in morgan_distinct_desc_labels:
    
    my_class = np.zeros(11)
    
    for num in elem.split(","):
        my_class[int(num)] = 1
        
    multi_class_output.append(my_class)

In [36]:
np.sum(multi_class_output, axis=0)

array([133.,  79., 155.,  80., 338., 122.,  24., 243., 381., 529., 569.])

In [37]:
scaler = MinMaxScaler()

morgan_distinct_desc_data = scaler.fit_transform(morgan_distinct_desc_data)

X_train, X_test, y_train, y_test = train_test_split(morgan_distinct_desc_data, multi_class_output, test_size=0.2)



In [38]:
np.sum(y_test, axis=0)/np.sum(y_train, axis=0)

array([0.23148148, 0.21538462, 0.25      , 0.21212121, 0.24723247,
       0.32608696, 0.26315789, 0.25906736, 0.25742574, 0.23887588,
       0.27008929])

In [39]:
multi_class_output = np.array(multi_class_output)

In [40]:
X_train, y_train, X_test, y_test = iterative_train_test_split(morgan_distinct_desc_data, multi_class_output, test_size = 0.2)


In [41]:
np.sum(y_test, axis=0)/np.sum(y_train, axis=0)

array([0.25471698, 0.25396825, 0.25      , 0.25      , 0.25185185,
       0.24489796, 0.26315789, 0.25257732, 0.24918033, 0.25059102,
       0.25054945])

In [42]:
clf = RandomForestClassifier(n_estimators=500, min_samples_split=5, max_depth=10, verbose=True)
clf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    2.2s finished


RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=500,
                       verbose=True)

In [43]:
y_pred = clf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.2s finished


In [44]:
y_pred[y_pred<0.5]  = 0
y_pred[y_pred>=0.5] = 1

In [45]:
for c in range(11):
    y_true_c = y_test[:,c]
    y_pred_c = y_pred[:,c]
    
    auc       = accuracy_score(y_true_c, y_pred_c)
    precision = precision_score(y_true_c, y_pred_c)
    recall    = recall_score(y_true_c, y_pred_c)
    
    print('Class '+str(c+1)+' statistics:')
    print('Accuracy %.4f, Precision %.4f, Recall %.4f\n' %(auc, precision, recall))

Class 1 statistics:
Accuracy 0.9375, Precision 0.0000, Recall 0.0000

Class 2 statistics:
Accuracy 0.9630, Precision 0.0000, Recall 0.0000

Class 3 statistics:
Accuracy 0.9329, Precision 1.0000, Recall 0.0645

Class 4 statistics:
Accuracy 0.9630, Precision 0.0000, Recall 0.0000

Class 5 statistics:
Accuracy 0.8426, Precision 0.0000, Recall 0.0000

Class 6 statistics:
Accuracy 0.9444, Precision 0.0000, Recall 0.0000

Class 7 statistics:
Accuracy 0.9884, Precision 0.0000, Recall 0.0000

Class 8 statistics:
Accuracy 0.9074, Precision 1.0000, Recall 0.1837

Class 9 statistics:
Accuracy 0.8819, Precision 0.9310, Recall 0.3553

Class 10 statistics:
Accuracy 0.8125, Precision 1.0000, Recall 0.2358

Class 11 statistics:
Accuracy 0.8102, Precision 0.9444, Recall 0.2982



/Users/gianluca/python_venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gianluca/python_venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gianluca/python_venv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/gianluca/python_venv/lib/python3.7/site-packages/sklearn/metrics/_classi

## 3. MACCS

In [46]:
maccs_desc = pd.read_csv('../data/new_descriptors/maccs_desc.txt', names=["fingerprint", "mol_id"], delimiter=" ")

In [47]:
maccs_desc_data = []

for i in range(len(maccs_desc['fingerprint'])):
    maccs_desc_data.append(np.array([int(j) for j in list(maccs_desc['fingerprint'].iloc[i])]).T)

In [48]:
maccs_desc_labels = []

for i in range(len(maccs_desc['mol_id'])):
    
    isFound = False
    
    if not "mol" in maccs_desc['mol_id'].iloc[i]:
        
        if len(maccs_desc['mol_id'].iloc[i]) == 3:
        
            maccs_desc['mol_id'].iloc[i] = "mol_0" + maccs_desc['mol_id'].iloc[i]
            
        else:
            
            maccs_desc['mol_id'].iloc[i] = "mol_" + maccs_desc['mol_id'].iloc[i]
            
        #print(maccs_desc['mol_id'].iloc[i])
    
    for j in range(len(mols_id)):
        if maccs_desc['mol_id'].iloc[i] == mols_id[j]:
            maccs_desc_labels.append(classes[j])
            isFound = True
            break
            
    if isFound == False:
        print("Here")
        print(maccs_desc['mol_id'].iloc[i])

In [49]:
# Convert classes to multi-class vector

multi_class_output = []

for elem in maccs_desc_labels:
    
    my_class = np.zeros(11)
    
    for num in elem.split(","):
        my_class[int(num)] = 1
        
    multi_class_output.append(my_class)

In [50]:
scaler = MinMaxScaler()

maccs_desc_data = scaler.fit_transform(maccs_desc_data)

X_train, X_test, y_train, y_test = train_test_split(maccs_desc_data, multi_class_output, test_size=0.2)








In [51]:
np.sum(y_test, axis=0)/np.sum(y_train, axis=0)

array([0.22118743, 0.24022346, 0.25125   , 0.25738397, 0.24270073,
       0.21917808, 0.23430962, 0.24613221, 0.234375  , 0.23333333,
       0.28967495])

In [52]:
multi_class_output = np.array(multi_class_output)

In [53]:
X_train, y_train, X_test, y_test = iterative_train_test_split(maccs_desc_data, multi_class_output, test_size = 0.2)



In [54]:
np.sum(y_test, axis=0)/np.sum(y_train, axis=0)

array([0.25029797, 0.31360947, 0.24968789, 0.25210084, 0.24954128,
       0.28365385, 0.32286996, 0.24964739, 0.24956063, 0.25      ,
       0.2502317 ])

In [55]:
clf = RandomForestClassifier(n_estimators=500, min_samples_split=5, max_depth=10, verbose=True)
clf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    4.5s finished


RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=500,
                       verbose=True)

In [56]:
y_pred = clf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.5s finished


In [57]:
y_pred[y_pred<0.5]  = 0
y_pred[y_pred>=0.5] = 1

In [58]:
for c in range(11):
    y_true_c = y_test[:,c]
    y_pred_c = y_pred[:,c]
    
    auc       = accuracy_score(y_true_c, y_pred_c)
    precision = precision_score(y_true_c, y_pred_c)
    recall    = recall_score(y_true_c, y_pred_c)
    
    print('Class '+str(c+1)+' statistics:')
    print('Accuracy %.4f, Precision %.4f, Recall %.4f\n' %(auc, precision, recall))

Class 1 statistics:
Accuracy 0.9473, Precision 0.8524, Recall 0.8524

Class 2 statistics:
Accuracy 0.9023, Precision 0.7558, Recall 0.4088

Class 3 statistics:
Accuracy 0.9320, Precision 0.9054, Recall 0.6700

Class 4 statistics:
Accuracy 0.9694, Precision 1.0000, Recall 0.4000

Class 5 statistics:
Accuracy 0.8870, Precision 0.7983, Recall 0.6838

Class 6 statistics:
Accuracy 0.9269, Precision 0.9211, Recall 0.2966

Class 7 statistics:
Accuracy 0.9541, Precision 1.0000, Recall 0.2500

Class 8 statistics:
Accuracy 0.8946, Precision 1.0000, Recall 0.2994

Class 9 statistics:
Accuracy 0.9142, Precision 0.9420, Recall 0.6866

Class 10 statistics:
Accuracy 0.8641, Precision 0.9449, Recall 0.6027

Class 11 statistics:
Accuracy 0.8997, Precision 0.8958, Recall 0.6370



## 4. Morgan

In [60]:
morgan_desc = pd.read_csv('../data/new_descriptors/morgan_desc.txt', names=["fingerprint", "mol_id"], delimiter=" ")

In [61]:
morgan_desc_data = []

for i in range(len(morgan_desc['fingerprint'])):
    morgan_desc_data.append(np.array([int(j) for j in list(morgan_desc['fingerprint'].iloc[i])]).T)

In [62]:
morgan_desc_labels = []

for i in range(len(morgan_desc['mol_id'])):
    
    isFound = False
    
    if not "mol" in morgan_desc['mol_id'].iloc[i]:
        
        if len(morgan_desc['mol_id'].iloc[i]) == 3:
        
            morgan_desc['mol_id'].iloc[i] = "mol_0" + morgan_desc['mol_id'].iloc[i]
            
        else:
            
            morgan_desc['mol_id'].iloc[i] = "mol_" + morgan_desc['mol_id'].iloc[i]
            
        #print(maccs_desc['mol_id'].iloc[i])
    
    for j in range(len(mols_id)):
        if morgan_desc['mol_id'].iloc[i] == mols_id[j]:
            morgan_desc_labels.append(classes[j])
            isFound = True
            break
            
    if isFound == False:
        print("Here")
        print(morgan_desc['mol_id'].iloc[i])

In [63]:
# Convert classes to multi-class vector

multi_class_output = []

for elem in morgan_desc_labels:
    
    my_class = np.zeros(11)
    
    for num in elem.split(","):
        my_class[int(num)] = 1
        
    multi_class_output.append(my_class)

In [64]:
multi_class_output = np.array(multi_class_output)

In [65]:
scaler = MinMaxScaler()

morgan_desc_data = scaler.fit_transform(morgan_desc_data)

X_train, X_test, y_train, y_test = train_test_split(morgan_desc_data, multi_class_output, test_size=0.2)



In [66]:
np.sum(y_test, axis=0)/np.sum(y_train, axis=0)

array([0.23557126, 0.2661597 , 0.25438596, 0.25738397, 0.24725275,
       0.24475524, 0.25      , 0.26934097, 0.25507502, 0.22923588,
       0.24791859])

In [67]:
X_train, y_train, X_test, y_test = iterative_train_test_split(morgan_desc_data, multi_class_output, test_size = 0.2)


In [68]:
np.sum(y_train, axis=0)

array([ 839.,  507.,  801.,  238., 1090.,  416.,  223.,  709., 1138.,
       1480., 1079.])

In [69]:
np.sum(y_test, axis=0)

array([210., 159., 200.,  60., 272., 118.,  72., 177., 284., 370., 270.])

In [70]:
np.sum(y_test, axis=0)/np.sum(y_train, axis=0)

array([0.25029797, 0.31360947, 0.24968789, 0.25210084, 0.24954128,
       0.28365385, 0.32286996, 0.24964739, 0.24956063, 0.25      ,
       0.2502317 ])

In [71]:
clf = RandomForestClassifier(n_estimators=500, min_samples_split=5, max_depth=10, verbose=True)
clf.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    7.0s finished


RandomForestClassifier(max_depth=10, min_samples_split=5, n_estimators=500,
                       verbose=True)

In [72]:
y_pred = clf.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.4s finished


In [73]:
y_pred[y_pred<0.5]  = 0
y_pred[y_pred>=0.5] = 1

In [74]:
for c in range(11):
    y_true_c = y_test[:,c]
    y_pred_c = y_pred[:,c]
    
    auc       = accuracy_score(y_true_c, y_pred_c)
    precision = precision_score(y_true_c, y_pred_c)
    recall    = recall_score(y_true_c, y_pred_c)
    
    print('Class '+str(c+1)+' statistics:')
    print('Accuracy %.4f, Precision %.4f, Recall %.4f\n' %(auc, precision, recall))

Class 1 statistics:
Accuracy 0.8945, Precision 0.9778, Recall 0.4190

Class 2 statistics:
Accuracy 0.8826, Precision 0.9200, Recall 0.1447

Class 3 statistics:
Accuracy 0.9319, Precision 1.0000, Recall 0.6000

Class 4 statistics:
Accuracy 0.9549, Precision 1.0000, Recall 0.1167

Class 5 statistics:
Accuracy 0.8391, Precision 0.9029, Recall 0.3419

Class 6 statistics:
Accuracy 0.9047, Precision 1.0000, Recall 0.0508

Class 7 statistics:
Accuracy 0.9677, Precision 1.0000, Recall 0.4722

Class 8 statistics:
Accuracy 0.8672, Precision 1.0000, Recall 0.1186

Class 9 statistics:
Accuracy 0.8613, Precision 0.9549, Recall 0.4472

Class 10 statistics:
Accuracy 0.8043, Precision 0.9930, Recall 0.3811

Class 11 statistics:
Accuracy 0.8102, Precision 0.9608, Recall 0.1815

